In [5]:
#!/usr/bin/env python3
"""
Monte-Carlo verification for Examples 3.3 and 3.4.

This script reproduces, in a simplified setting, the risk calculations
for several experimental designs discussed in the writeup:

Example 3.3:
    - 6 components (i = 1,...,6)
    - Population size N = 1000, with K = 200 ones in each component
    - Sample n = 100 ballots without replacement
    - Local test for H_i: R_i = {π_i < 14}, where π_i is the number of 1s in the sample
    - Global test: reject H⋆ iff *all* six local tests reject (intersection of R_i)

Example 3.4:
    - 9 components (i = 1,...,9)
    - Same N, K, n
    - Three different global designs using local tests:

      Design A (naive union, "tempting but invalid"):
          R_i = {π_i < 14} at level ≈ 0.039
          Global: reject H⋆ iff at least one R_i occurs (union of R_i)

      Design B (risk-budgeted union, "solution 1."):
          R_i' = {π_i ≤ 10} at level ≈ 0.004
          Global: reject H⋆ iff at least one R_i' occurs (union of R_i')

      Design C (single-edge audit, "solution 2."):
          Only test the first local null H_1 using R_1 = {π_1 < 14}
          All other components are ignored; we reject H⋆ iff R_1 occurs.

For each design we estimate the number of Type I errors in a fixed
number of trials (default 50_000 when run as a script) and summarize
the results in a pandas DataFrame that can be exported to LaTeX.
"""

from __future__ import annotations

from math import comb
from dataclasses import dataclass, asdict
from typing import List

import numpy as np
import pandas as pd


# ---------- exact hypergeometric computations ----------

def hypergeom_cdf(k: int, K: int, N: int, n: int) -> float:
    """
    CDF P[X <= k] for X ~ Hypergeom(N, K, n).
    """
    num = 0
    for x in range(0, k + 1):
        num += comb(K, x) * comb(N - K, n - x)
    den = comb(N, n)
    return num / den


def exact_probs() -> None:
    """
    Compute exact single-component probabilities used in the text.
    """
    N = 1000
    K = 200
    n = 100

    p_pi_lt_14 = hypergeom_cdf(13, K, N, n)   # P(π < 14) = P(π <= 13)
    p_pi_le_10 = hypergeom_cdf(10, K, N, n)   # P(π <= 10)

    print("=== Exact hypergeometric probabilities (single component) ===")
    print(f"P(π < 14 | P_i = 200)  = P(π <= 13) ≈ {p_pi_lt_14:.6f}")
    print(f"P(π <= 10 | P_i = 200)               ≈ {p_pi_le_10:.6f}")
    print()


# ---------- helper dataclass for collecting results ----------

@dataclass
class DesignResult:
    example: str
    design: str
    trials: int
    type1_errors: int

    @property
    def empirical_risk(self) -> float:
        return self.type1_errors / self.trials

    def as_dict(self):
        d = asdict(self)
        d["empirical_risk"] = self.empirical_risk
        return d


# ---------- Monte Carlo for Example 3.3 ----------

def simulate_example_3_3(
    trials: int = 50_000,
    seed: int | None = 0,
) -> DesignResult:
    """
    Monte-Carlo experiment for Example 3.3.

    For each trial:
      - Draw π_1,...,π_6 independently from Hypergeom(N=1000, K=200, n=100).
      - Local rejection rule for each i: R_i = {π_i < 14}.

    Global design:
      - Reject H⋆ iff ALL 6 local tests reject (intersection of R_i).
    """
    rng = np.random.default_rng(seed)
    N = 1000
    K = 200
    n = 100
    threshold_13 = 13  # π < 14

    pis = rng.hypergeometric(K, N - K, n, size=(trials, 6))
    local_reject = pis <= threshold_13            # shape (trials, 6)
    global_reject = local_reject.all(axis=1)      # intersection over i

    num_type1 = int(global_reject.sum())

    print("=== Example 3.3 Monte-Carlo ===")
    print(f"Trials: {trials}")
    print("Local size per coordinate (P(false reject H_i)):")
    print(local_reject.mean(axis=0))
    print(f"Global size P(false reject H⋆) ≈ {num_type1 / trials:.6e}")
    print()

    return DesignResult(
        example="3.3",
        design="intersection_all_6_pi<14",
        trials=trials,
        type1_errors=num_type1,
    )


# ---------- Monte Carlo for Example 3.4 ----------

def simulate_example_3_4(
    trials: int = 50_000,
    seed: int | None = 1,
) -> List[DesignResult]:
    """
    Monte-Carlo experiments for Example 3.4.

    For each trial:
      - Draw π_1,...,π_9 independently from Hypergeom(N=1000, K=200, n=100).

    Designs:

      A: naive union of local tests R_i = {π_i < 14}
         -> reject H⋆ if ANY local test rejects.

      B: risk-budgeted union ("solution 1.")
         local tests R_i' = {π_i <= 10}
         -> reject H⋆ if ANY local test rejects.

      C: single-edge audit ("solution 2.")
         Only test H_1 with R_1 = {π_1 < 14}
         -> reject H⋆ if R_1 occurs; other coordinates ignored.
    """
    rng = np.random.default_rng(seed)
    N = 1000
    K = 200
    n = 100
    thresh_13 = 13  # π < 14
    thresh_10 = 10  # π <= 10

    pis = rng.hypergeometric(K, N - K, n, size=(trials, 9))

    local_lt14 = pis <= thresh_13  # for π_i < 14
    local_le10 = pis <= thresh_10  # for π_i <= 10

    results: List[DesignResult] = []

    # Design A: naive union, π_i < 14
    global_naive = local_lt14.any(axis=1)
    num_naive = int(global_naive.sum())
    print("=== Example 3.4, Design A (naive union, π_i < 14) ===")
    print(f"Trials: {trials}")
    print(f"Estimated P(false reject H⋆) ≈ {num_naive / trials:.6f}")
    print()

    results.append(
        DesignResult(
            example="3.4",
            design="A_naive_union_pi<14",
            trials=trials,
            type1_errors=num_naive,
        )
    )

    # Design B: risk-budgeted union, π_i <= 10 ("solution 1.")
    global_budgeted = local_le10.any(axis=1)
    num_budgeted = int(global_budgeted.sum())
    print("=== Example 3.4, Design B (union, π_i <= 10) ===")
    print(f"Trials: {trials}")
    print(f"Estimated P(false reject H⋆) ≈ {num_budgeted / trials:.6f}")
    print()

    results.append(
        DesignResult(
            example="3.4",
            design="B_union_pi<=10",
            trials=trials,
            type1_errors=num_budgeted,
        )
    )

    # Design C: single-edge audit, only first local null ("solution 2.")
    # Reject H⋆ iff R_1 = {π_1 < 14} occurs; other coordinates ignored.
    first_edge_reject = local_lt14[:, 0]  # only use component i=1
    num_single = int(first_edge_reject.sum())
    print("=== Example 3.4, Design C (single-edge, first component only) ===")
    print(f"Trials: {trials}")
    print(f"Estimated P(false reject H⋆) ≈ {num_single / trials:.6f}")
    print()

    results.append(
        DesignResult(
            example="3.4",
            design="C_single_edge_first_pi<14",
            trials=trials,
            type1_errors=num_single,
        )
    )

    return results


# ---------- main: run everything and summarize ----------

def main(trials: int = 50_000) -> None:
    exact_probs()

    res_3_3 = simulate_example_3_3(trials=trials, seed=0)
    res_3_4_list = simulate_example_3_4(trials=trials, seed=1)

    all_results = [res_3_3] + res_3_4_list
    df = pd.DataFrame([r.as_dict() for r in all_results])

    print("=== Summary DataFrame (Type I errors over trials) ===")
    print(df)
    print()

    print("=== LaTeX table (df.to_latex) ===")
    # You can customize float_format, column formatting, etc. as needed.
    print(df.to_latex(index=False, float_format=lambda x: f"{x:.6f}"))


#if __name__ == "__main__":
main()


=== Exact hypergeometric probabilities (single component) ===
P(π < 14 | P_i = 200)  = P(π <= 13) ≈ 0.038955
P(π <= 10 | P_i = 200)               ≈ 0.003993

=== Example 3.3 Monte-Carlo ===
Trials: 50000
Local size per coordinate (P(false reject H_i)):
[0.0398  0.0391  0.03956 0.03902 0.03946 0.0383 ]
Global size P(false reject H⋆) ≈ 0.000000e+00

=== Example 3.4, Design A (naive union, π_i < 14) ===
Trials: 50000
Estimated P(false reject H⋆) ≈ 0.300700

=== Example 3.4, Design B (union, π_i <= 10) ===
Trials: 50000
Estimated P(false reject H⋆) ≈ 0.035140

=== Example 3.4, Design C (single-edge, first component only) ===
Trials: 50000
Estimated P(false reject H⋆) ≈ 0.039180

=== Summary DataFrame (Type I errors over trials) ===
  example                     design  trials  type1_errors  empirical_risk
0     3.3   intersection_all_6_pi<14   50000             0         0.00000
1     3.4        A_naive_union_pi<14   50000         15035         0.30070
2     3.4             B_union_pi<=10 